In [ ]:
import pandas as pd

In [ ]:
%%capture
!pip install gdown

In [ ]:
!gdown "https://drive.google.com/uc?id=1i-sTZzhYwf_ShhTCFNGyMNugaVIkMix6"


Downloading...
From: https://drive.google.com/uc?id=1i-sTZzhYwf_ShhTCFNGyMNugaVIkMix6
To: /content/Eval.zip
100% 1.29M/1.29M [00:00<00:00, 26.3MB/s]


In [ ]:
! unzip /content/Eval.zip

Archive:  /content/Eval.zip
   creating: Eval/
   creating: Eval/model_base_7/
  inflating: Eval/model_base_7/base_model_outputs.csv  
  inflating: Eval/model_base_7/metrics.json  
  inflating: Eval/model_base_7/desktop.ini  
   creating: Eval/model_base_1/
  inflating: Eval/model_base_1/metric base model 1.json  
  inflating: Eval/model_base_1/base_model_outputs.csv  
  inflating: Eval/model_base_1/desktop.ini  
   creating: Eval/model_ft_1/
  inflating: Eval/model_ft_1/fine tune base model.json  
  inflating: Eval/model_ft_1/fine_tuned_outputs.csv  
  inflating: Eval/model_ft_1/desktop.ini  
   creating: Eval/model_ft_7/
  inflating: Eval/model_ft_7/fine_tuned_outputs.csv  
  inflating: Eval/model_ft_7/metrics.json  
  inflating: Eval/model_ft_7/desktop.ini  
   creating: Eval/model_base_8/
  inflating: Eval/model_base_8/base_model_outputs.csv  
  inflating: Eval/model_base_8/metrics.json  
  inflating: Eval/model_base_8/desktop.ini  
   creating: Eval/model_base_5/
   creating: Eval

In [ ]:
# prompt: import bert score, bleu, rougr
%%capture
!pip install bert-score
!pip install evaluate
!pip install rouge_score

In [ ]:
import bert_score
from evaluate import load
import pandas as pd

In [ ]:
import evaluate

In [ ]:
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import evaluate

bleu = evaluate.load("bleu")

predictions = ["the cat is on the mat"]
references = [["there is a cat on the mat"]]

# BLEU-1
result_bleu1 = bleu.compute(predictions=predictions, references=references, max_order=1)
print(f"BLEU-1: {result_bleu1['bleu']:.4f}")

# BLEU-2
result_bleu2 = bleu.compute(predictions=predictions, references=references, max_order=2)
print(f"BLEU-2: {result_bleu2['bleu']:.4f}")

# BLEU-3
result_bleu3 = bleu.compute(predictions=predictions, references=references, max_order=3)
print(f"BLEU-3: {result_bleu3['bleu']:.4f}")

# BLEU-4 (default)
result_bleu4 = bleu.compute(predictions=predictions, references=references)
print(f"BLEU-4: {result_bleu4['bleu']:.4f}")


BLEU-1: 0.7054
BLEU-2: 0.4887
BLEU-3: 0.3697
BLEU-4: 0.0000


In [ ]:
pd.options.display.max_colwidth = 500

Function Compute Bleu score


In [ ]:
import evaluate
import pandas as pd

# Load evaluator sekali saja
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

def compute_bleu_per_row(predictions, references):
    """
    Menghitung BLEU 1-4 untuk setiap pasangan prediksi dan referensi.
    Return: list of dict, tiap dict {'bleu1':..., 'bleu2':..., 'bleu3':..., 'bleu4':...}
    """
    all_scores = []
    for pred, ref in zip(predictions, references):
        scores = {}
        # ref harus list of references, meskipun 1 referensi
        ref_list = [ref] if isinstance(ref, str) else ref
        for n in range(1, 5):
            score = bleu.compute(predictions=[pred], references=[ref_list], max_order=n, smooth=True)
            scores[f'bleu{n}'] = score['bleu']
        all_scores.append(scores)
    return all_scores

Function Compoute Rouge Score

In [ ]:
def compute_rouge_per_row(predictions, references):
    rouge = evaluate.load("rouge")
    scores = []
    for pred, ref in zip(predictions, references):
        score = rouge.compute(predictions=[pred], references=[ref])
        # Ambil langsung fmeasure-nya (hasil evaluate adalah dict float)
        scores.append({
            "rouge1": score["rouge1"],
            "rouge2": score["rouge2"],
            "rougeL": score["rougeL"],
            "rougeLSum": score["rougeLsum"]
        })
    return scores


Bert score multiligual bert

In [ ]:
import evaluate

def compute_bertscore_per_row(predictions, references):
    """
    Menghitung BERTScore per kalimat untuk Bahasa Indonesia menggunakan library evaluate.

    Parameters:
    - predictions: List[str], list kalimat yang dihasilkan model.
    - references: List[List[str]], list referensi yang diharapkan (nested list, satu referensi per kalimat).

    Returns:
    - List of Dicts berisi precision, recall, dan F1 untuk setiap kalimat.
    """
    # Muat evaluator
    bertscore = evaluate.load("bertscore")

    # Pastikan referensi adalah list of strings (flatten)
    flat_references = [ref[0] for ref in references]

    # Hitung skor
    results = bertscore.compute(
        predictions=predictions,
        references=flat_references,
        lang="id"  # Bahasa Indonesia
    )

    # Gabungkan hasil per kalimat
    scores_per_row = []
    for i in range(len(predictions)):
        scores_per_row.append({
            "prediction": predictions[i],
            "reference": flat_references[i],
            "bert_precision": results["precision"][i],
            "bert_recall": results["recall"][i],
            "bert_f1": results["f1"][i],
        })

    return scores_per_row

# Contoh penggunaan
predictions = ["kucing itu duduk di atas tikar"]
references = [["seekor kucing duduk di tikar"]]

scores = compute_bertscore_per_row(predictions, references)

print("BERTScore per row:")
for i, score in enumerate(scores):
    print(f"Row {i+1}:")
    print(f"  Prediction: {score['prediction']}")
    print(f"  Reference : {score['reference']}")
    print(f"  Precision : {score['bert_precision']:.4f}")
    print(f"  Recall    : {score['bert_recall']:.4f}")
    print(f"  F1        : {score['bert_f1']:.4f}")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_REMOVED_BY_SCRIPT' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_REMOVED_BY_SCRIPT]` or `pip install hf_REMOVED_BY_SCRIPT`


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

BERTScore per row:
Row 1:
  Prediction: kucing itu duduk di atas tikar
  Reference : seekor kucing duduk di tikar
  Precision : 0.7809
  Recall    : 0.7824
  F1        : 0.7817


# Model 4

## Base Model 4

In [ ]:
base_4 = pd.read_csv('/content/Eval/model_base_4/content/eval_basemodel_20250525_034008/base_model_outputs.csv')
base_4.sample(1)

,instruction,input,reference,generated,bert_precision,bert_recall,bert_f1
91,"Jawab pertanyaan medis berikut dengan bahasa yang mudah dipahami. Pastikan respons mencakup: penjelasan kondisi, informasi penting, dan saran yang relevan.","Obat untuk gastritis akibat infeksi bakteri H pylori. sore dok, saya positif h pylori. dikasih obay azytromicin 500 moga(1x) pagi. amixillin 500 moga (4x), lanzoprasole 2x diminum 1/2 jam sebelum makan. apa sudah benar dok obatnya. trmksh","pylori. beberapa gejala yang dialami diantaranya: • nyeri pada daerah ulu hati • nyeri semakin bertambah jika ada makanan yang masuk • mual • muntah • tubuh terasa lemas prinsip pengobatan untuk gastritis ada dua, yaitu dengan farmakoterapi (obat) dan modifikasi gaya hidup. melalui farmakoterapi digunakan kombinasi 3 obat yaitu omperazole/lansoprazole+klaritromisin/tetrasiklin+amoksisilin/metronidazol dapat digunakan jika penyebab gastritisnya adalah kuman h. pylori. perubahan gaya hidup jug...","Gastritis akibat infeksi bakteri H pylori. sore dok, saya positif h pylori. dikasih obat azytromicin 500 mg (1x) pagi. amixillin 500 mg (4x), lanzoprasole 2x diminum 1/2 jam sebelum makan. apa sudah benar dok obatnya. trmksh\n\n### Explanation:\nThe instruction requires the responder to answer the question with a language that is easy to understand. Make sure the response includes: explanation of the condition, important information, and relevant advice.",0.638424,0.594769,0.615824


In [ ]:
len(base_4)

100

In [ ]:
base_4.isnull().sum()

,0
instruction,0
input,0
reference,0
generated,1
bert_precision,0
bert_recall,0
bert_f1,0


In [ ]:
base_4.fillna('empty', inplace=True)

In [ ]:
# Buang baris yang memiliki nan pada kolom generated atau reference
df_clean = base_4.dropna(subset=['generated', 'reference']).copy()

preds = df_clean['generated'].tolist()
refs = df_clean['reference'].tolist()

# Pastikan references berbentuk list of list untuk BLEU
refs_for_bleu = [[ref] for ref in refs]

bleu_scores = compute_bleu_per_row(preds, refs_for_bleu)
rouge_scores = compute_rouge_per_row(preds, refs)

# Buat dict untuk dataframe baru
data_new = {}

for n in range(1, 5):
    data_new[f'bleu{n}'] = [score[f'bleu{n}'] for score in bleu_scores]

data_new['rouge1'] = [score['rouge1'] for score in rouge_scores]
data_new['rouge2'] = [score['rouge2'] for score in rouge_scores]
data_new['rougeL'] = [score['rougeL'] for score in rouge_scores]
data_new['rougeLSum'] = [score['rougeLSum'] for score in rouge_scores]

In [ ]:
bertscore_eval_scores = compute_bertscore_per_row(preds, refs_for_bleu)

In [ ]:
df_bert_score = pd.DataFrame(bertscore_eval_scores)
df_bert_score

,prediction,reference,bert_precision,bert_recall,bert_f1
0,"Maaf, dokter tidak tersedia pada hari ini. Berikut ini adalah informasi untuk menjawab pertanyaan anda.\n\nMaag adalah penyakit yang terjadi akibat luka pada perut. Maag terjadi ketika luka di perut mengalami infeksi dan menyebabkan perdarahan. Maag terjadi akibat infeksi virus atau bakteri. Maag juga dapat terjadi akibat kondisi lain seperti kanker, cedera perut, atau infeksi yang disebabkan oleh bakteri.\n\nGejala maag yang sering muncul adalah sakit perut, muntah, dan mual. Gejala-gejala ...","usus halus ataupun kerongkongan yang dapat disebabkan kondisi berikut : faktor psikis, yakni kondisi stres emosional akibat pengunaan obat antinyeri non steroid (nsaid) akibat perlukaan pada lambung (tukak lambung/peptik) infeksi bakteri helicobacter pylori (gastritis) dsb sakit maag yang kronis atau berlangsung berat dapat menyebabkan perlukaan pada lambung dan sebaliknya. pada perlukaan yang berat dapat terjadi perforasi (pecahnya) jaringan lambung sehingga menimbulkan gejala batuk/muntah ...",0.705103,0.661096,0.682391
1,"Jawab pertanyaan medis berikut dengan bahasa yang mudah dipahami. Pastikan respons mencakup: penjelasan kondisi, informasi penting, dan saran yang relevan.\n\n### Input:\nsolusi untuk mengat",konstipasi adalah keadaan tidak dapat buang air besar secara teratur atau tidak dapat buang air besar sama sekali. konstipasi dapat disebabkan oleh: kurang mimun kurang konsumsi serat kurang aktif bergerak dan berolahraga kelebihan atau kekurangan berat badan kebiasaan menunda buang air besar cemas atau depresi penyumbatan usus gangguan saraf usus efek samping obat-obatan tertentu dsb. sebaiknya anda berkonsultasi dengan dokter di mana dokter akan menganalisis gejala dan melakukan pemeriksaa...,0.644507,0.582390,0.611876
2,"Assalamu'alaikum,\n\nPerut sakit, mual dan bersendawa berlebihan. Saya ingin tany dok.. istri saya merasakan sakit perut, kepala pusing di sertai mual2 dan bersendawa yang berlebih.. kira-kira apa penyebab nya dan bagaimana solusi mengatasinya. Terima kasih.\n\nTerima kasih atas pertanyaan medis Anda. Apakah perut sakit Anda berhubungan dengan mual, pusing, dan bersendawa? Jika ya, ini merupakan gejala dari penyakit apapun. Saya akan mencoba menjawab pertanyaan Anda sebaik mungkin dengan inf...","penyebab sakit perut, mual dan bersendawa mungkin disebabkan oleh sin, yang gejala-gejalanya adalah: sering bersendawa tidak nyaman di perut, kembung mual penurunan nafsu makan dada terasa seperti terbakar penyebab dispepsia ini diantaranya: refluks asam lambung (gerd) ulkus peptikum intoleransi laktosa batu empedu dan radang empedu depresi dan kecemasan efek samping obat-obatan, misalnya: antibiotik, aspirin konsumsi kopi,dulu untuk memastikan diagnosis penyakit tersebut istri anda dapat be...",0.634713,0.627972,0.631325
3,"Jawab pertanyaan medis berikut dengan bahasa yang mudah dipahami. Pastikan respons mencakup: penjelasan kondisi, informasi penting, dan saran yang relev","sebaiknya anda berkonsultasi dengan dokter di mana dokter akan menganalisis gejala yang anda alami dan melakukan pemeriksaan fisik yang diperlukan. pemeriksaan penunjang seperti tes darah, tes urine, usg mungkin akan dokter anjurkan. terapi yang sesuai akan dokter berikan.anjuran yang dapat anda lakukan: konsumsi makanan bersih dan bergizi secara teratur minum air putih dalam jumlah cukup minimal 10 gelas perhari hindari kebiasaan menahan buang air kecil dan buang air besar manajemen stres s...",0.661312,0.605073,0.631944
4,"Kondisi alergi makanan seafood dapat ditangani dengan beberapa cara, antara lain:\n1. Mencegah kontak dengan bahan alergi. Dengan cara ini, Anda dapat memastikan bahwa tidak ada bahan alergi yang masuk ke tubuh Anda. Hal ini dapat dilakukan dengan cara menghindari makanan yang mengandung bahan alergi.\n2. Mengonsumsi obat anti-inflamasi. Obat anti-inflamasi dapat membantu mengurangi gejala alergi, seperti gatal-gatal dan bengkak. Obat anti-inflamasi yang sering 

In [ ]:
df_scores = pd.DataFrame(data_new)
df_scores = df_scores * 100
df_scores = pd.concat([df_bert_score, df_scores], axis=1)
df_scores.head(2)

,prediction,reference,bert_precision,bert_recall,bert_f1,bleu1,bleu2,bleu3,bleu4,rouge1,rouge2,rougeL,rougeLSum
0,"Maaf, dokter tidak tersedia pada hari ini. Berikut ini adalah informasi untuk menjawab pertanyaan anda.\n\nMaag adalah penyakit yang terjadi akibat luka pada perut. Maag terjadi ketika luka di perut mengalami infeksi dan menyebabkan perdarahan. Maag terjadi akibat infeksi virus atau bakteri. Maag juga dapat terjadi akibat kondisi lain seperti kanker, cedera perut, atau infeksi yang disebabkan oleh bakteri.\n\nGejala maag yang sering muncul adalah sakit perut, muntah, dan mual. Gejala-gejala ...","usus halus ataupun kerongkongan yang dapat disebabkan kondisi berikut : faktor psikis, yakni kondisi stres emosional akibat pengunaan obat antinyeri non steroid (nsaid) akibat perlukaan pada lambung (tukak lambung/peptik) infeksi bakteri helicobacter pylori (gastritis) dsb sakit maag yang kronis atau berlangsung berat dapat menyebabkan perlukaan pada lambung dan sebaliknya. pada perlukaan yang berat dapat terjadi perforasi (pecahnya) jaringan lambung sehingga menimbulkan gejala batuk/muntah ...",0.705103,0.661096,0.682391,17.169811,6.495710,2.518932,1.319653,19.973369,3.471295,11.717710,13.581891
1,"Jawab pertanyaan medis berikut dengan bahasa yang mudah dipahami. Pastikan respons mencakup: penjelasan kondisi, informasi penting, dan saran yang relevan.\n\n### Input:\nsolusi untuk mengat",konstipasi adalah keadaan tidak dapat buang air besar secara teratur atau tidak dapat buang air besar sama sekali. konstipasi dapat disebabkan oleh: kurang mimun kurang konsumsi serat kurang aktif bergerak dan berolahraga kelebihan atau kekurangan berat badan kebiasaan menunda buang air besar cemas atau depresi penyumbatan usus gangguan saraf usus efek samping obat-obatan tertentu dsb. sebaiknya anda berkonsultasi dengan dokter di mana dokter akan menganalisis gejala dan melakukan pemeriksaa...,0.644507,0.582390,0.611876,0.774941,0.248743,0.172068,0.144252,4.705882,0.000000,4.705882,4.705882


In [ ]:
# Calculate the mean of the numerical columns only
numerical_cols = ['bert_precision', 'bert_recall', 'bert_f1', 'bleu1', 'bleu2', 'bleu3', 'bleu4', 'rouge1', 'rouge2', 'rougeL', 'rougeLSum']
mean_scores = df_scores[numerical_cols].mean()

print("\nRata-rata Skor:")
mean_scores


Rata-rata Skor:


,0
bert_precision,0.628678
bert_recall,0.600173
bert_f1,0.613041
bleu1,8.522933
bleu2,2.880937
bleu3,1.473571
bleu4,0.980875
rouge1,12.311867
rouge2,1.299153
rougeL,7.692550


In [ ]:
import os
os.makedirs('Hasil Eval revisi')
mean_scores.to_csv('Hasil Eval revisi/eval_basemodel_4_mean.csv')

In [ ]:
df_scores = pd.concat([base_4[['input']], df_scores], axis=1)
df_scores.head(2)

,input,prediction,reference,bert_precision,bert_recall,bert_f1,bleu1,bleu2,bleu3,bleu4,rouge1,rouge2,rougeL,rougeLSum
0,"Solusi atasi sakit maag yang tak kunjung sembuh. Saya memiliki keluhan selama 2 minggu belakangan ada masalah di pencernaan. Saya sudah berobat klinik 2 kali tapi diagnosanya sama yaitu saya menderita maag. Oleh dokter yang menangani, saya hanya diberi antasida, omeprazole, dan ranitidin. Saya sudah habiskan obatnya, pola makan pun saya jaga dengan tidak mengkonsumsi makanan pedas, asam, dan kafein namun sampai saat ini tidak ada perubahan. Selama 2 minggu saya sakit berat badan saya turun 1...","Maaf, dokter tidak tersedia pada hari ini. Berikut ini adalah informasi untuk menjawab pertanyaan anda.\n\nMaag adalah penyakit yang terjadi akibat luka pada perut. Maag terjadi ketika luka di perut mengalami infeksi dan menyebabkan perdarahan. Maag terjadi akibat infeksi virus atau bakteri. Maag juga dapat terjadi akibat kondisi lain seperti kanker, cedera perut, atau infeksi yang disebabkan oleh bakteri.\n\nGejala maag yang sering muncul adalah sakit perut, muntah, dan mual. Gejala-gejala ...","usus halus ataupun kerongkongan yang dapat disebabkan kondisi berikut : faktor psikis, yakni kondisi stres emosional akibat pengunaan obat antinyeri non steroid (nsaid) akibat perlukaan pada lambung (tukak lambung/peptik) infeksi bakteri helicobacter pylori (gastritis) dsb sakit maag yang kronis atau berlangsung berat dapat menyebabkan perlukaan pada lambung dan sebaliknya. pada perlukaan yang berat dapat terjadi perforasi (pecahnya) jaringan lambung sehingga menimbulkan gejala batuk/muntah ...",0.705103,0.661096,0.682391,17.169811,6.495710,2.518932,1.319653,19.973369,3.471295,11.717710,13.581891
1,"solusi untuk mengatasi konstipasi. dok kalo bab feses saya enggak keras, cair tapi usus saya enggak mau untuk ngeluarinnya. kenapa ya dok?","Jawab pertanyaan medis berikut dengan bahasa yang mudah dipahami. Pastikan respons mencakup: penjelasan kondisi, informasi penting, dan saran yang relevan.\n\n### Input:\nsolusi untuk mengat",konstipasi adalah keadaan tidak dapat buang air besar secara teratur atau tidak dapat buang air besar sama sekali. konstipasi dapat disebabkan oleh: kurang mimun kurang konsumsi serat kurang aktif bergerak dan berolahraga kelebihan atau kekurangan berat badan kebiasaan menunda buang air besar cemas atau depresi penyumbatan usus gangguan saraf usus efek samping obat-obatan tertentu dsb. sebaiknya anda berkonsultasi dengan dokter di mana dokter akan menganalisis gejala dan melakukan pemeriksaa...,0.644507,0.582390,0.611876,0.774941,0.248743,0.172068,0.144252,4.705882,0.000000,4.705882,4.705882


In [ ]:
df_scores.to_csv('Hasil Eval revisi/eval_basemodel_4.csv')

## Fine Tune Model 4

In [ ]:
def evaluate_generation_scores(data):
    # Buang baris yang memiliki NaN pada kolom generated atau reference
    df_clean = data.dropna(subset=['generated', 'reference']).copy()

    preds = df_clean['generated'].tolist()
    refs = df_clean['reference'].tolist()

    # Pastikan references berbentuk list of list untuk BLEU
    refs_for_bleu = [[ref] for ref in refs]

    # Hitung BLEU dan ROUGE per baris
    bleu_scores = compute_bleu_per_row(preds, refs_for_bleu)
    rouge_scores = compute_rouge_per_row(preds, refs)

    # Hitung BERTScore per baris
    bert_scores = compute_bertscore_per_row(preds, refs_for_bleu)

    # Siapkan dictionary untuk skor
    data_new = {}

    for n in range(1, 5):
        data_new[f'bleu{n}'] = [score[f'bleu{n}'] for score in bleu_scores]

    data_new['rouge1'] = [score['rouge1'] for score in rouge_scores]
    data_new['rouge2'] = [score['rouge2'] for score in rouge_scores]
    data_new['rougeL'] = [score['rougeL'] for score in rouge_scores]
    data_new['rougeLSum'] = [score['rougeLSum'] for score in rouge_scores]

    # Tambahkan BERTScore per row
    data_new['bert_precision'] = [score['bert_precision'] for score in bert_scores]
    data_new['bert_recall'] = [score['bert_recall'] for score in bert_scores]
    data_new['bert_f1'] = [score['bert_f1'] for score in bert_scores]

    # Konversi ke DataFrame dan skalakan ke persen
    df_scores = pd.DataFrame(data_new)
    df_scores = df_scores * 100

    # Tambahkan kolom input, reference, dan generated
    df_scores['input'] = df_clean['input'].values
    df_scores['reference'] = df_clean['reference'].values
    df_scores['generated'] = df_clean['generated'].values

    # Hitung rata-rata skor
    mean_scores = df_scores.drop(columns=['input', 'reference', 'generated']).mean()

    return df_scores, mean_scores


In [ ]:
ft_model_4 = pd.read_csv('/content/Eval/model_ft_4/content/eval_basemodel_20250525_044500/fine_tuned_outputs.csv')
ft_model_4.sample(1)

,instruction,input,reference,generated,bert_precision,bert_recall,bert_f1
84,"Jawab pertanyaan medis berikut dengan bahasa yang mudah dipahami. Pastikan respons mencakup: penjelasan kondisi, informasi penting, dan saran yang relevan.","Susah BAB disertai sakit perut. Dok,saya mau tanya. Hari senin malam saya nyeri tulang dan demam karena kedinginan,lalu saya minumin jus dan minuman jahe. Namun selasa pagi saya nyeri tulang disertai diare, selama 1 hari saya BAB 6x dan malamnya saya muntah berupa cairan bening sangat pahit 2x. Rabu pagi saya bawa ke dokter, saya di beri obat. Namun setelah mengkonsumsi obat, saya jadi susah BAB. Namun perut saya melilit seperti mau BAB dan itu sering (sekitar 4-5x dalam sehari). Setiap kali...","kondisi klinis seperti tifus , ileus 2. efek dari obat dimana anda harus berkonsultasi dengan dokter anda mengenai detail obat yang diberikan 3. infeksi saluran cerna konsultasikan dengan dokter anda mengenai kondisi yang anda alami untuk penanganan dan informasi lebih lanjut. pemeriksaan fisik dapat membantu untuk menilai kondisi yang anda alami. selain itu, pemeriksaan lain yang dapat membantu seperti foto rontgen, feses jika diperlukan. hal yang dapat anda lakukan yakni menghindari sement...","keluhan yang anda alami kemungkinan merupakan gejala dari penyakit maag. penyakit maag merupakan suatu kondisi dimana terjadi peradangan pada dinding lambung. beberapa penyebab yang dapat menyebabkan terjadinya penyakit maag diantaranya: infeksi bakteri helicobacter pylori efek samping dari penggunaan obat-obatan tertentu seperti obat anti inflamasi non steroid (oains) seperti asam mefenamat, ibuprofen, atau naproxen konsumsi makanan yang mengandung asam dan pedas kelelahan stres psikologis ...",0.68871,0.723709,0.705776


In [ ]:
ft_model_4.isnull().sum()

,0
instruction,0
input,0
reference,0
generated,0
bert_precision,0
bert_recall,0
bert_f1,0


In [ ]:
df_ft, mean_ft = evaluate_generation_scores(ft_model_4)

In [ ]:
df_ft.head(2)

,bleu1,bleu2,bleu3,bleu4,rouge1,rouge2,rougeL,rougeLSum,bert_precision,bert_recall,bert_f1,input,reference,generated
0,34.997256,16.489537,7.171295,3.347295,34.394904,6.823028,14.012739,14.012739,74.620163,72.101730,73.339337,"Solusi atasi sakit maag yang tak kunjung sembuh. Saya memiliki keluhan selama 2 minggu belakangan ada masalah di pencernaan. Saya sudah berobat klinik 2 kali tapi diagnosanya sama yaitu saya menderita maag. Oleh dokter yang menangani, saya hanya diberi antasida, omeprazole, dan ranitidin. Saya sudah habiskan obatnya, pola makan pun saya jaga dengan tidak mengkonsumsi makanan pedas, asam, dan kafein namun sampai saat ini tidak ada perubahan. Selama 2 minggu saya sakit berat badan saya turun 1...","usus halus ataupun kerongkongan yang dapat disebabkan kondisi berikut : faktor psikis, yakni kondisi stres emosional akibat pengunaan obat antinyeri non steroid (nsaid) akibat perlukaan pada lambung (tukak lambung/peptik) infeksi bakteri helicobacter pylori (gastritis) dsb sakit maag yang kronis atau berlangsung berat dapat menyebabkan perlukaan pada lambung dan sebaliknya. pada perlukaan yang berat dapat terjadi perforasi (pecahnya) jaringan lambung sehingga menimbulkan gejala batuk/muntah ...","maag atau biasa disebut sebagai gastritis adalah suatu penyakit yang disebabkan oleh adanya peradangan pada dinding lambung. gastritis terjadi karena lapisan lambung mengalami iritasi, peradangan, atau pengikisan oleh asam lambung yang diproduksi oleh sel-sel dalam lambung. gastritis dapat disebabkan oleh beberapa faktor berikut: pola makan yang buruk, seperti terlambat makan, makan makanan yang pedas, asam, berlemak, atau yang menghasilkan banyak gas, seperti kol, kacang-kacangan, minuman b..."
1,54.430380,37.239244,25.657258,17.182931,54.861111,24.475524,43.055556,43.055556,79.042411,78.676975,78.859270,"solusi untuk mengatasi konstipasi. dok kalo bab feses saya enggak keras, cair tapi usus saya enggak mau untuk ngeluarinnya. kenapa ya dok?",konstipasi adalah keadaan tidak dapat buang air besar secara teratur atau tidak dapat buang air besar sama sekali. konstipasi dapat disebabkan oleh: kurang mimun kurang konsumsi serat kurang aktif bergerak dan berolahraga kelebihan atau kekurangan berat badan kebiasaan menunda buang air besar cemas atau depresi penyumbatan usus gangguan saraf usus efek samping obat-obatan tertentu dsb. sebaiknya anda berkonsultasi dengan dokter di mana dokter akan menganalisis gejala dan melakukan pemeriksaa...,"konstipasi dapat disebabkan oleh beberapa hal berikut ini: kebiasaan mengabaikan keinginan untuk buang air besar kurang minum kurang konsumsi serat kurang aktif dan jarang berolahraga penurunan berat badan yang mendadak efek samping obat-obatan tertentu penyakit medis tertentu seperti penyakit crohn, penyumbatan usus, diabetes, gangguan tiroid, kanker usus, dsb. untuk memastikan penyebab dari konstipasi yang anda alami, maka sebaiknya anda memeriksakan diri secara langsung ke dokter. dokter ..."


In [ ]:
mean_ft

,0
bleu1,26.152145
bleu2,12.577292
bleu3,6.848691
bleu4,4.149928
rouge1,30.238819
rouge2,6.806361
rougeL,15.668782
rougeLSum,15.668782
bert_precision,69.596678
bert_recall,69.194938


### Menyimpan Hasil Evaluasi

In [ ]:
ft_model_4.to_csv('Hasil Eval revisi/eval_ft_model_4.csv')

In [ ]:
mean_ft.to_csv('Hasil Eval revisi/eval_ft_model_4_mean.csv')

In [ ]:
!zip -r /content/Hasil_Eval_revisi-4.zip /content/Hasil\ Eval\ revisi

  adding: content/Hasil Eval revisi/ (stored 0%)
  adding: content/Hasil Eval revisi/eval_ft_model_4_mean.csv (deflated 40%)
  adding: content/Hasil Eval revisi/eval_basemodel_4_mean.csv (deflated 38%)
  adding: content/Hasil Eval revisi/eval_basemodel_4.csv (deflated 74%)
  adding: content/Hasil Eval revisi/eval_ft_model_4.csv (deflated 73%)
